<a href="https://colab.research.google.com/github/filipediniz/Coursera_Capstone/blob/master/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install requests_html
!pip install pixiedust
!pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

/bin/bash: conda: command not found


In [0]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import pixiedust
import requests
from requests_html import HTMLSession
import re as regex

import datetime as dt
from pandas_datareader import data

from IPython.display import clear_output

import pickle

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [0]:
#%%pixie_debugger

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')    
try:
    DF = pd.read_html(ret.text, decimal=",", thousands='.', encoding='utf-8')[0]
    #DF.columns = DF.iloc[0]
    #DF = DF.reset_index()
except:
    print('Página sem dados')
    

In [0]:
from bs4 import BeautifulSoup

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/Irvine,_California')    
try:    
    soup = BeautifulSoup(ret.text, 'html.parser')
except:
    print('Página sem dados')

In [50]:
html_objs = soup.find_all("h4")
villages = ''
for html_obj in html_objs:
  print(html_obj.text)
  if html_obj.text == 'Villages':    
    villages = html_obj.next_element.next_element.next_element.next_element.next_element.next_element.next_element
    break
neighborhoods = []
for village in list((list(villages.children)[1]).children):
  if village.name != None:    
    neighborhoods.append(village.text.split(' (')[0].split('[')[0])    

Villages


 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [51]:
DF = pd.DataFrame()
DF['Neighborhood'] = neighborhoods
DF

,Neighborhood
0,El Camino Glen
1,College Park
2,The Colony
3,Columbus Grove
4,Cypress Village
5,Deerfield
6,East Irvine
7,El Camino Real
8,Greentree
9,Irvine Groves


4. Submit a link to your Notebook on your Github repository. (10 marks)

# END OF PART 1

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [52]:
#@title
import geocoder # import geocoder
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

DF['Latitude'] =  None
DF['Longitude'] = None

for r in range(0, DF.shape[0]):
  print('Current row: {}'.format(r))
  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  g = None
  print('Attempting to retrieve coordinates for address {}...'.format('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood'])))
  geolocator = Nominatim(user_agent="irvine_explorer")
  g = geolocator.geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )          
  #g = do_geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )            
  if (type(g) != type(None)):    
    print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
    DF.at[r, 'Latitude'] = g.latitude
    DF.at[r, 'Longitude'] = g.longitude    

Current row: 0
Attempting to retrieve coordinates for address El Camino Glen, Irvine, California...
Latitude = 33.6960782; longitude = -117.790868
Current row: 1
Attempting to retrieve coordinates for address College Park, Irvine, California...
Latitude = 33.7049508; longitude = -117.79327516984964
Current row: 2
Attempting to retrieve coordinates for address The Colony, Irvine, California...
Latitude = 33.75507555; longitude = -117.80374469402437
Current row: 3
Attempting to retrieve coordinates for address Columbus Grove, Irvine, California...
Latitude = 33.7045487; longitude = -117.7637001
Current row: 4
Attempting to retrieve coordinates for address Cypress Village, Irvine, California...
Latitude = 33.6835428; longitude = -117.75398984807549
Current row: 5
Attempting to retrieve coordinates for address Deerfield, Irvine, California...
Latitude = 33.6929179; longitude = -117.7907331
Current row: 6
Attempting to retrieve coordinates for address East Irvine, Irvine, California...
Lati

In [53]:
DF

,Neighborhood,Latitude,Longitude
0,El Camino Glen,33.6961,-117.791
1,College Park,33.705,-117.793
2,The Colony,33.7551,-117.804
3,Columbus Grove,33.7045,-117.764
4,Cypress Village,33.6835,-117.754
5,Deerfield,33.6929,-117.791
6,East Irvine,33.6639,-117.743
7,El Camino Real,33.6942,-117.785
8,Greentree,33.6976,-117.79
9,Irvine Groves,33.7055,-117.783


# END OF PART 2

Explore and cluster the neighborhoods in irvine. You can decide to work with only boroughs that contain the word irvine and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

First it is necessary to drop all rows from which contains NaN values.

Then, in order to limit the analysis only for irvine area, one neeeds to filter the rows keeping only the boroughs whose name contains "irvine".

In [54]:
DF.dropna(inplace=True)
DF.reset_index()

,index,Neighborhood,Latitude,Longitude
0,0,El Camino Glen,33.6961,-117.791
1,1,College Park,33.705,-117.793
2,2,The Colony,33.7551,-117.804
3,3,Columbus Grove,33.7045,-117.764
4,4,Cypress Village,33.6835,-117.754
5,5,Deerfield,33.6929,-117.791
6,6,East Irvine,33.6639,-117.743
7,7,El Camino Real,33.6942,-117.785
8,8,Greentree,33.6976,-117.79
9,9,Irvine Groves,33.7055,-117.783


In [55]:
DF

,Neighborhood,Latitude,Longitude
0,El Camino Glen,33.6961,-117.791
1,College Park,33.705,-117.793
2,The Colony,33.7551,-117.804
3,Columbus Grove,33.7045,-117.764
4,Cypress Village,33.6835,-117.754
5,Deerfield,33.6929,-117.791
6,East Irvine,33.6639,-117.743
7,El Camino Real,33.6942,-117.785
8,Greentree,33.6976,-117.79
9,Irvine Groves,33.7055,-117.783


In order to visualize irvine area within a interactive map, it requires the latitude and longitude coordinates. They were retrieved using the Nominatim package.

In [56]:
address = 'Irvine'

geolocator = Nominatim(user_agent="irvine_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Irvine are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Irvine are 33.6856969, -117.8259819.


Then a map in created using the Folium package.

In [62]:
# create map of irvine using latitude and longitude values
map_irvine = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(DF['Latitude'], DF['Longitude'], DF['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_irvine)  
    
map_irvine

In order to use the Foursquare API, it is necessary to input client ID, client secret, version and limit.

In [63]:
CLIENT_ID = '4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI' # your Foursquare ID
CLIENT_SECRET = 'UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI
CLIENT_SECRET:UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44


This function aims at listing interesting places that are located within 1600 units radius from the point defined by some pair of latitude / longitude values.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

This list is the result from using the function above.

In [72]:
irvine_venues = getNearbyVenues(names=DF['Neighborhood'],
                                   latitudes=DF['Latitude'],
                                   longitudes=DF['Longitude']
                                  )

El Camino Glen
College Park
The Colony
Columbus Grove
Cypress Village
Deerfield
East Irvine
El Camino Real
Greentree
Irvine Groves
Harvard Square
Laguna Altura
Lambert Ranch
Northpark
Northpark Square
Northwood
Oak Creek
Old Towne Irvine
Orangetree
Orchard Hills
Park Lane
Parkcrest
Parkside
Pavilion Park
Portola Springs
Quail Hill
Racquet Club
The Ranch
Rancho San Joaquin
Rosegate
Stonegate
Shady Canyon
Turtle Ridge
Turtle Rock
University Hills
University Park
University Town Center
Walnut
West Irvine
Westpark
The Willows
Windwood
Woodbridge
Woodbury
Woodbury East


The resulting dataframe, which, besides the Neighborhood name, also contains latitude, longitude from both the neighborhoods themselves and the venues, including their category.

In [73]:
#irvine_venues.set_index(list(irvine_venues)[0], inplace=True)
irvine_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,El Camino Glen,33.696078,-117.790868,Lovely Nails,33.696466,-117.798373,Spa
1,El Camino Glen,33.696078,-117.790868,All That Barbecue,33.696866,-117.798111,Korean Restaurant
2,El Camino Glen,33.696078,-117.790868,Trader Joe's,33.706082,-117.788254,Grocery Store
3,El Camino Glen,33.696078,-117.790868,The Habit Burger Grill,33.704707,-117.786916,Burger Joint
4,El Camino Glen,33.696078,-117.790868,Sam Woo Seafood Restaurant,33.695909,-117.798085,Chinese Restaurant


The next step would be to format the above dataframe using the one hot encoding.

In [74]:
# one hot encoding
irvine_onehot = pd.get_dummies(irvine_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
irvine_onehot['Neighborhood'] = irvine_venues['Neighborhood'] 

# move neighborhood column to the first column
irvine_onehot = pd.concat([irvine_onehot['Neighborhood'].to_frame(), irvine_onehot.drop(columns=['Neighborhood'], axis=1)], axis=1)
irvine_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,...,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,South American Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,El Camino Glen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The problem is that this format is very sparse making the dimension of the dataframe to be very large.

In [75]:
irvine_onehot.shape

(2857, 207)

In [76]:
irvine_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,El Camino Glen,33.696078,-117.790868,Lovely Nails,33.696466,-117.798373,Spa
1,El Camino Glen,33.696078,-117.790868,All That Barbecue,33.696866,-117.798111,Korean Restaurant
2,El Camino Glen,33.696078,-117.790868,Trader Joe's,33.706082,-117.788254,Grocery Store
3,El Camino Glen,33.696078,-117.790868,The Habit Burger Grill,33.704707,-117.786916,Burger Joint
4,El Camino Glen,33.696078,-117.790868,Sam Woo Seafood Restaurant,33.695909,-117.798085,Chinese Restaurant
...,...,...,...,...,...,...,...
2852,Woodbury East,33.691785,-117.741812,center terrace park,33.700358,-117.731464,Park
2853,Woodbury East,33.691785,-117.741812,Bosque Park,33.687350,-117.727604,Park
2854,Woodbury East,33.691785,-117.741812,Cadence Park,33.685279,-117.727209,Park
2855,Woodbury East,33.691785,-117.741812,Olive Grove Square,33.701797,-117.753260,Park
